# Average antibody escape across `polyclonal` models
This notebook aggregates and averages the antibody escape computed across multiple fit `polyclonal` models to different libraries, replicates, etc.

First, import Python modules:

In [ ]:
import os
import pickle

import pandas as pd

Get parameterized variables from [papermill](https://papermill.readthedocs.io/):

In [ ]:
# papermill parameters cell (tagged as `parameters`)
antibody = None
escape_avg_method = None
avg_pickle = None
selection_groups_dict = None

In [ ]:
# This is just a temporary completed parameters cell like papermill will make when called with antibody REGN10933
selection_groups_dict = {
    "LibA_2022-02-10a_thaw-3_REGN10933_1": {
        "date": "2022-02-10a",
        "library": "LibA",
        "replicate": 1,
        "virus_batch": "thaw-3",
        "pickle_file": "results/polyclonal_fits/LibA_2022-02-10a_thaw-3_REGN10933_1.pickle",
    },
    "LibA_2022-02-10a_thaw-3_REGN10933_2": {
        "date": "2022-02-10a",
        "library": "LibA",
        "replicate": 2,
        "virus_batch": "thaw-3",
        "pickle_file": "results/polyclonal_fits/LibA_2022-02-10a_thaw-3_REGN10933_2.pickle",
    },
    "LibB_2022-03-02_thaw-3_REGN10933_1": {
        "date": "2022-03-02",
        "library": "LibB",
        "replicate": 1,
        "virus_batch": "thaw-3",
        "pickle_file": "results/polyclonal_fits/LibB_2022-03-02_thaw-3_REGN10933_1.pickle",
    },
}
antibody = "REGN10933"
escape_avg_method = "median"
avg_pickle = "results/antibody_escape/REGN10933.pickle"

# another temporary command to get in right subdir
os.chdir("../test_example")

Convert `selection_groups` into a data frame and get all of the pickled models:

In [ ]:
selection_groups = pd.DataFrame.from_dict(selection_groups_dict, orient="index")

assert all(map(os.path.isfile, selection_groups["pickle_file"]))

selection_groups